In [1]:
# !pip install plotly
# !pip install seaborn
# !pip install matplotlib
# !pip install transformers
# !pip install tokenizers
# !pip install scikit-learn
# !pip install tensorflow
# !pip install pandas
# !pip install numpy
# !pip install keras_tuner


from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#
<a id="data-prep"></a>
<div style="background-color: #000D5B; color: white; text-align: center; padding: 6px 0 22px 0">
    <h3 style="background-color: #000D5B; color: white; text-align: left">RMIT School of Computer Science and Technology</h3>
    <br/>
    <h1>COSC3007: Deep Learning</h1>
    <h2>Assignment 2: Stance Twitter Sentiment Analysis and Detection </h2>
    <h2> [2] MODELLING AND MODEL EVALUATIONS </h2>

</div>

# Import Libs

In [2]:
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, BatchNormalization, Bidirectional, Flatten
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers.legacy import Adam
from sklearn.metrics import classification_report
import numpy as np

import numpy as np

# [1] Load Data

In [3]:
train_df = pd.read_csv("./drive/MyDrive/Colab Notebooks/StanceDataset/processed_train_augumented_vast.csv", encoding = "ISO-8859-1", engine="python").drop(columns=['Unnamed: 0'])
train_df = train_df.sample(frac=1, random_state=42).reset_index(drop=True)
train_ori = train_df.copy()

test_df = pd.read_csv("./drive/MyDrive/Colab Notebooks/StanceDataset/processed_test_augumented_vast.csv", encoding = "ISO-8859-1", engine="python").drop(columns=['Unnamed: 0'])

In [4]:
print(len(train_ori))
print(len(test_df))

16391
1956


# [2] Prepare label and fit data

## Split test and val

In [5]:
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42, stratify=train_df['Stance'])

## Tweets

In [6]:
from transformers import TFRobertaModel, RobertaTokenizer
import numpy as np

# Initialize the BERT tokenizer
MAX_LENGTH = 512
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
roberta_model = TFRobertaModel.from_pretrained('roberta-base')

# Assuming train_texts, val_texts, test_texts are your datasets
train_processed_tweets = train_df['Tweet'].tolist()
val_processed_tweets = val_df['Tweet'].tolist()
test_processed_tweets = test_df['Tweet'].tolist()

all_processed_tweets = train_processed_tweets + val_processed_tweets + test_processed_tweets
tokenized_tweets = [tokenizer.encode(tweet, add_special_tokens=True) for tweet in all_processed_tweets]
MAX_LENGTH = max(len(tweet) for tweet in tokenized_tweets)
MAX_LENGTH = 128

print("Maximum length across all datasets:", MAX_LENGTH)

target_names = {
    0: "Hillary Clinton",
    1: "Legalization of Abortion",
    2: "Atheism",
    3: "Climate Change is a Real Concern",
    4: "Feminist Movement",
    5: "Donald Trump",
    6: "Joe Biden",
    7: "Barack Obama",
    8: "Kamala Harris",
    9: "Bernie Sanders",
    10: "Vu",
}

def bert_tokenize(texts, targets, max_length=MAX_LENGTH):
    input_ids = []
    attention_masks = []

    for idx, text in enumerate(texts):
        target_name = targets[idx]
        # target_name = target_names.get(targetID, "")
        formatted_text = f"<s> {target_name} </s></s> {text} </s>"
        encoded = tokenizer.encode_plus(
            formatted_text,
            add_special_tokens=True,
            max_length=max_length,
            truncation=True,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='tf',
        )

        # Flatten the tensors and remove the extra dimension
        input_ids.append(tf.squeeze(encoded['input_ids'], axis=0))
        attention_masks.append(tf.squeeze(encoded['attention_mask'], axis=0))

    return np.array(input_ids), np.array(attention_masks)

# Assuming you have lists of preprocessed texts for training, validation, and test sets
train_input_ids, train_attention_masks = bert_tokenize(train_df['Tweet'],  train_df['Target'].to_list())
val_input_ids, val_attention_masks = bert_tokenize(val_df['Tweet'], val_df['Target'].to_list())
test_input_ids, test_attention_masks = bert_tokenize(test_df['Tweet'], test_df['Target'].to_list())

print(train_input_ids.shape)
print(val_input_ids.shape)
print(test_input_ids.shape)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'roberta.embeddings.position_ids', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- T

Maximum length across all datasets: 128
(13112, 128)
(3279, 128)
(1956, 128)


## Stances

In [7]:
def categorized_label(df, label_name):
    label_encoder = LabelEncoder()
    encoded_labels = label_encoder.fit_transform(df[label_name])
    categorical_labels = to_categorical(encoded_labels)
    return categorical_labels

In [8]:
train_labels = categorized_label(train_df, "Stance")
test_labels = categorized_label(test_df, "Stance")
val_labels = categorized_label(val_df, "Stance")

# [3] Create Model

In [9]:
from keras import backend as K
from tensorflow.keras.optimizers.legacy import Adam
from hyperopt import hp
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
# import keras_tuner as kt

In [10]:
def f1_score_class(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))

    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

def f1_score(y_true, y_pred):
    # Convert predictions to one-hot format
    y_pred_one_hot = K.one_hot(K.argmax(y_pred), num_classes=3)

    # Calculate F1 score for each class
    f1s = [f1_score_class(y_true[:, i], y_pred_one_hot[:, i]) for i in range(3)]

    # Average F1 scores across all classes
    return K.mean(K.stack(f1s), axis=0)

Set up call backs and learning rates

In [11]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint


# Define the EarlyStopping and ReduceLROnPlateau callbacks
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    verbose=1,
    restore_best_weights=True,
)

lr_scheduler = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=5,
    min_lr=1e-5,
    verbose=1,
)

# Define the ModelCheckpoint callback
checkpoint_path = "roberta_best_model.h5"
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,    # Only save a model if 'val_loss' has improved
    monitor='val_loss',     # Monitor 'val_loss' during training
    mode='min',             # The model is saved when 'val_loss' is minimized
    verbose=1)

# Combine all callbacks in a list
callbacks = [
    early_stopping,
    lr_scheduler,
    model_checkpoint_callback
]

In [12]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, BatchNormalization, Bidirectional, Flatten, Input, GlobalMaxPooling1D
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers.legacy import Adam
# import keras_tuner as kt
from transformers import TFRobertaModel

def build_model(drop, regr, lr):
    bert_model = TFRobertaModel.from_pretrained('roberta-base')
    bert_model.trainable = False

    # Define model input (BERT expects two inputs: input_ids and attention_mask)
    input_ids = Input(shape=(MAX_LENGTH,), dtype=tf.int32, name='input_ids')
    attention_mask = Input(shape=(MAX_LENGTH,), dtype=tf.int32, name='attention_mask')

    # Get BERT embeddings
    bert_output = bert_model(input_ids, attention_mask=attention_mask)[0]  # Use the pooled output here

    # Tuning the number of units in the first biLSTM layer
    x = Bidirectional(LSTM(units=64,  dropout=drop, recurrent_dropout=drop,return_sequences=True,kernel_regularizer=tf.keras.regularizers.l2(regr))) (bert_output)

    x = GlobalMaxPooling1D()(x)

    # Dense layer with tunable units
    x = Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(regr)) (x)

    # Tunable dropout rate for the second layer
    x = Dropout(drop)(x)

    # Final Layer
    x = Dense(3, activation='softmax')(x)

    # Tuning the learning rate for the optimizer
    optimizer = Adam(learning_rate=lr)
    model = Model(inputs=[input_ids, attention_mask], outputs=x)
    model.compile(loss='CategoricalCrossentropy', optimizer=optimizer, metrics=['categorical_accuracy',f1_score])
    model.summary()

    return model



Recreate the best model with more training time

In [13]:
history = {}
inital_model = build_model(0.8, 0.0005, 0.001)
history['tuned_stance_model_custom'] = inital_model.fit([train_input_ids, train_attention_masks], train_labels,
                    epochs=1000,
                    validation_data=([val_input_ids, val_attention_masks], val_labels),
                    batch_size=256,
                    callbacks=callbacks)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'roberta.embeddings.position_ids', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- T

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(None, 128)]                0         []                            
                                                                                                  
 attention_mask (InputLayer  [(None, 128)]                0         []                            
 )                                                                                                
                                                                                                  
 tf_roberta_model_1 (TFRobe  TFBaseModelOutputWithPooli   1246456   ['input_ids[0][0]',           
 rtaModel)                   ngAndCrossAttentions(last_   32         'attention_mask[0][0]']      
                             hidden_state=(None, 128, 7                                       

In [14]:
results = inital_model.evaluate([test_input_ids, test_attention_masks], test_labels)
print('Test accuracy:', results)

62/62 [==============================] - 8s 135ms/step - loss: 0.8633 - categorical_accuracy: 0.6355 - f1_score: 0.4178
Test accuracy: [0.8632698059082031, 0.6354805827140808, 0.41783618927001953]


In [15]:
# Get model predictions
predictions = inital_model.predict([test_input_ids, test_attention_masks])

# Convert predictions to class labels
predicted_labels = np.argmax(predictions, axis=1)

# Assuming test_labels are one-hot encoded, convert them to class labels
true_labels = np.argmax(test_labels, axis=1)

# Generate classification report
report = classification_report(true_labels, predicted_labels)
print(report)

62/62 [==============================] - 12s 133ms/step
              precision    recall  f1-score   support

           0       0.64      0.82      0.72      1014
           1       0.62      0.41      0.50       452
           2       0.62      0.45      0.52       490

    accuracy                           0.64      1956
   macro avg       0.63      0.56      0.58      1956
weighted avg       0.63      0.64      0.62      1956



In [22]:
def load_classification_rp_per_target(model):
    targets = test_df.Target.unique()
    metrics_df = pd.DataFrame()
    for target in targets:
        extracted_df = test_df[test_df['Target'] == target]
        input_ids, attention_masks = bert_tokenize(extracted_df['Tweet'], extracted_df['Target'].to_list())
        labels = categorized_label(extracted_df, "Stance")

        # Convert one-hot encoded labels to single integer class labels
        true_labels = np.argmax(labels, axis=1)

        # Get model predictions
        predictions = model.predict([input_ids, attention_masks])

        # Convert predictions to class labels
        predicted_labels = np.argmax(predictions, axis=1)

        # Generate a classification report
        report = classification_report(true_labels, predicted_labels, target_names=['AGAINST', 'FAVOR', 'NONE'])
        target_name = target
        # if target == 0:
        #     target_name = "Hillary Clinton"
        # if target == 1:
        #     target_name = "Legalization of Abortion"
        # if target == 2:
        #     target_name = "Atheism"
        # if target == 3:
        #     target_name = "Climate Change is a Real Concern"
        # if target == 4:
        #     target_name = "Feminist Movement"
        # if target == 5:
        #     target_name = "Donald Trump"

        print('Target: ', target_name)
        print(report)
        print("-----------------------------------------", "\n")

        # Flatten the report into a single row and add target name
        report = classification_report(true_labels, predicted_labels, target_names=['AGAINST', 'FAVOR', 'NONE'], output_dict=True)

        # Extract weighted f1, macro f1, and accuracy
        report_data = {
            'Target': target_name,
            'Weighted F1': report['weighted avg']['f1-score'],
            'Macro F1': report['macro avg']['f1-score'],
            'Accuracy': report['accuracy']
        }

        # Append to the DataFrame
        metrics_df = metrics_df.append(report_data, ignore_index=True)
    metrics_df.set_index('Target', inplace=True)

    return metrics_df

In [23]:
# Use the function to get the metrics DataFrame
model_metrics_df = load_classification_rp_per_target(inital_model)
model_metrics_df

7/7 [==============================] - 1s 137ms/step
Target:  Atheism
              precision    recall  f1-score   support

     AGAINST       0.90      0.67      0.77       160
       FAVOR       0.50      0.69      0.58        32
        NONE       0.44      0.89      0.59        28

    accuracy                           0.70       220
   macro avg       0.61      0.75      0.64       220
weighted avg       0.78      0.70      0.72       220

----------------------------------------- 



<ipython-input-22-653adaea9b0f>:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append(report_data, ignore_index=True)


6/6 [==============================] - 1s 135ms/step
Target:  Climate Change is a Real Concern
              precision    recall  f1-score   support

     AGAINST       0.00      0.00      0.00        11
       FAVOR       0.88      0.59      0.70       123
        NONE       0.40      0.97      0.56        35

    accuracy                           0.63       169
   macro avg       0.42      0.52      0.42       169
weighted avg       0.72      0.63      0.63       169

----------------------------------------- 



<ipython-input-22-653adaea9b0f>:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append(report_data, ignore_index=True)


9/9 [==============================] - 1s 134ms/step
Target:  Feminist Movement
              precision    recall  f1-score   support

     AGAINST       0.79      0.60      0.68       183
       FAVOR       0.36      0.55      0.44        58
        NONE       0.57      0.73      0.64        44

    accuracy                           0.61       285
   macro avg       0.57      0.63      0.59       285
weighted avg       0.67      0.61      0.62       285

----------------------------------------- 



...

9/9 [==============================] - 1s 132ms/step
Target:  Legalization of Abortion
              precision    recall  f1-score   support

     AGAINST       0.82      0.67      0.74       189
       FAVOR       0.47      0.52      0.49        46
        NONE       0.43      0.73      0.55        45

    accuracy                           0.65       280
   macro avg       0.58      0.64      0.59       280
weighted avg       0.70      0.65      0.67       280

----------------------------------------- 



<ipython-input-22-653adaea9b0f>:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append(report_data, ignore_index=True)


23/23 [==============================] - 3s 132ms/step
Target:  Donald Trump
              precision    recall  f1-score   support

     AGAINST       0.56      0.64      0.60       299
       FAVOR       0.78      0.33      0.46       148
        NONE       0.68      0.80      0.74       260

    accuracy                           0.64       707
   macro avg       0.68      0.59      0.60       707
weighted avg       0.65      0.64      0.62       707

----------------------------------------- 



<ipython-input-22-653adaea9b0f>:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append(report_data, ignore_index=True)


,Weighted F1,Macro F1,Accuracy
Target,,,
Atheism,0.716913,0.644736,0.700000
Climate Change is a Real Concern,0.627630,0.421474,0.627219
Feminist Movement,0.624757,0.585496,0.610526
Hillary Clinton,0.781171,0.746208,0.783051
Legalization of Abortion,0.666327,0.592381,0.653571
Donald Trump,0.622621,0.600994,0.635078


[4] Fine Tune

In [18]:
for layer in inital_model.layers:
    layer.trainable = True
   # Tuning the learning rate for the optimizer
inital_model.compile(optimizer=tf.keras.optimizers.legacy.Adam(1e-5),
                  loss='categorical_crossentropy',
                  metrics=['categorical_accuracy', f1_score])

history['fine_tuned_bert'] = inital_model.fit([train_input_ids, train_attention_masks], train_labels,
                    epochs=30,
                    validation_data=([val_input_ids, val_attention_masks], val_labels),
                    batch_size=32,
                    callbacks=callbacks)

Epoch 1/30


410/410 [==============================] - ETA: 0s - loss: 0.8556 - categorical_accuracy: 0.6289 - f1_score: 0.6080
Epoch 1: saving model to roberta_best_model.h5
410/410 [==============================] - 353s 809ms/step - loss: 0.8556 - categorical_accuracy: 0.6289 - f1_score: 0.6080 - val_loss: 0.7461 - val_categorical_accuracy: 0.6905 - val_f1_score: 0.6877 - lr: 1.0000e-05
Epoch 2/30
410/410 [==============================] - ETA: 0s - loss: 0.7742 - categorical_accuracy: 0.6839 - f1_score: 0.6717
Epoch 2: saving model to roberta_best_model.h5
410/410 [==============================] - 329s 801ms/step - loss: 0.7742 - categorical_accuracy: 0.6839 - f1_score: 0.6717 - val_loss: 0.7194 - val_categorical_accuracy: 0.7048 - val_f1_score: 0.6971 - lr: 1.0000e-05
Epoch 3/30
410/410 [==============================] - ETA: 0s - loss: 0.7212 - categorical_accuracy: 0.7208 - f1_score: 0.7145
Epoch 3: saving model to roberta_best_model.h5
410/410 [==============================] - 329s 802ms

In [24]:
# Use the function to get the metrics DataFrame
model_metrics_df = load_classification_rp_per_target(inital_model)
model_metrics_df

7/7 [==============================] - 1s 133ms/step
Target:  Atheism
              precision    recall  f1-score   support

     AGAINST       0.90      0.67      0.77       160
       FAVOR       0.50      0.69      0.58        32
        NONE       0.44      0.89      0.59        28

    accuracy                           0.70       220
   macro avg       0.61      0.75      0.64       220
weighted avg       0.78      0.70      0.72       220

----------------------------------------- 



<ipython-input-22-653adaea9b0f>:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append(report_data, ignore_index=True)


6/6 [==============================] - 1s 127ms/step
Target:  Climate Change is a Real Concern
              precision    recall  f1-score   support

     AGAINST       0.00      0.00      0.00        11
       FAVOR       0.88      0.59      0.70       123
        NONE       0.40      0.97      0.56        35

    accuracy                           0.63       169
   macro avg       0.42      0.52      0.42       169
weighted avg       0.72      0.63      0.63       169

----------------------------------------- 



<ipython-input-22-653adaea9b0f>:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append(report_data, ignore_index=True)


9/9 [==============================] - 1s 135ms/step
Target:  Feminist Movement
              precision    recall  f1-score   support

     AGAINST       0.79      0.60      0.68       183
       FAVOR       0.36      0.55      0.44        58
        NONE       0.57      0.73      0.64        44

    accuracy                           0.61       285
   macro avg       0.57      0.63      0.59       285
weighted avg       0.67      0.61      0.62       285

----------------------------------------- 



...

9/9 [==============================] - 1s 132ms/step
Target:  Legalization of Abortion
              precision    recall  f1-score   support

     AGAINST       0.82      0.67      0.74       189
       FAVOR       0.47      0.52      0.49        46
        NONE       0.43      0.73      0.55        45

    accuracy                           0.65       280
   macro avg       0.58      0.64      0.59       280
weighted avg       0.70      0.65      0.67       280

----------------------------------------- 



<ipython-input-22-653adaea9b0f>:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append(report_data, ignore_index=True)


23/23 [==============================] - 3s 131ms/step
Target:  Donald Trump
              precision    recall  f1-score   support

     AGAINST       0.56      0.64      0.60       299
       FAVOR       0.78      0.33      0.46       148
        NONE       0.68      0.80      0.74       260

    accuracy                           0.64       707
   macro avg       0.68      0.59      0.60       707
weighted avg       0.65      0.64      0.62       707

----------------------------------------- 



<ipython-input-22-653adaea9b0f>:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append(report_data, ignore_index=True)


,Weighted F1,Macro F1,Accuracy
Target,,,
Atheism,0.716913,0.644736,0.700000
Climate Change is a Real Concern,0.627630,0.421474,0.627219
Feminist Movement,0.624757,0.585496,0.610526
Hillary Clinton,0.781171,0.746208,0.783051
Legalization of Abortion,0.666327,0.592381,0.653571
Donald Trump,0.622621,0.600994,0.635078


In [20]:
# Get model predictions
predictions = inital_model.predict([test_input_ids, test_attention_masks])

# Convert predictions to class labels
predicted_labels = np.argmax(predictions, axis=1)

# Assuming test_labels are one-hot encoded, convert them to class labels
true_labels = np.argmax(test_labels, axis=1)

# Generate classification report
report = classification_report(true_labels, predicted_labels)
print(report)

62/62 [==============================] - 8s 134ms/step
              precision    recall  f1-score   support

           0       0.73      0.67      0.70      1014
           1       0.62      0.50      0.55       452
           2       0.59      0.81      0.68       490

    accuracy                           0.66      1956
   macro avg       0.65      0.66      0.64      1956
weighted avg       0.67      0.66      0.66      1956



In [21]:
inital_model.save('roberta_model_66.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
